In [11]:
from bs4 import BeautifulSoup
import requests
import psycopg2
import re

conn = psycopg2.connect(database='solidot', user='python', password='passw0rd3pyth4n', host='127.0.0.1', port='5432')
db = conn.cursor()
db.execute('''CREATE TABLE IF NOT EXISTS guazi \
           (id INT PRIMARY KEY    NOT NULL,
           city      TEXT         NOT NULL,
           prise     TEXT         NOT NULL,
           context   TEXT,
           href      TEXT);''')
conn.commit()

head = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.19 Safari/537.36'}
city_list = []
ids = 0
www = 'http://www.guazi.com/'
rest = requests.get(www, headers=head)
soup1 = BeautifulSoup(rest.text, 'html.parser')
all_citys = len(soup1.select('div.header > div.hd-top.clearfix > div.c2city > div > div > dl > dt'))
for i in range(2,all_citys):
    city = soup1.select('div.header > div.hd-top.clearfix > div.c2city > div > div > dl > dd')[i]
    for j in range(len(city.findAll('a'))):
        shi = re.findall('/.+?/', str(city.findAll('a')[j]))
        city_list.append(shi)
#x=soup.select('div.c2city dd a')
#for city in x:
#    print city['href']

for p in range(len(city_list)):
    for page in range(1,8):
        if page == 1:
            html = 'http://www.guazi.com' + str(city_list[p][0]) + 'buy/'
        else:
            html = 'http://www.guazi.com' + str(city_list[p][0]) + 'buy/o' + str(page)        
#        print html
        res = requests.get(html, headers=head)
        soup = BeautifulSoup(res.text, 'html.parser')
        titles = soup.select('div.list > ul > li > div > p.infoBox > a')
        prices = soup.select('div.list > ul > li > div > p.priType-s > span:nth-of-type(1) > i') 
        times = len(soup.select('div.list > ul > li > div > p.infoBox > a'))
        for i in range(times):
            prise = prices[i].text.strip()
            context = titles[i].text
            href = 'http://www.guazi.com' + titles[i]['href']
            ids += 1
            db.execute("INSERT INTO guazi VALUES (%s, %s, %s, %s, %s)", (ids, city_list[p][0], prise, context, href))
            conn.commit()

conn.commit()
conn.close()
print("OK")


OK
